In [1]:
!pip3 install fastapi pandas torch transformers uvicorn google-cloud-storage scikit-learn


In [ ]:

from fastapi import FastAPI, UploadFile, HTTPException
import pandas as pd
import torch
import json
import io
from transformers import BartTokenizer

In [ ]:

# Load the new JSON files and inspect their structure

with open("./datasets/input_old_model/us-central1_eeg-test_dodadqada_dodadqada_saved_data_input_embeddings_12.json", 'r') as file:
    input_embeddings_data = json.load(file)

with open("./datasets/input_old_model/us-central1_eeg-test_dodadqada_dodadqada_saved_data_input_mask_invert_12.json", 'r') as file:
    input_mask_invert_data = json.load(file)

with open("./datasets/input_old_model/us-central1_eeg-test_dodadqada_dodadqada_saved_data_input_masks_12.json", 'r') as file:
    input_masks_data = json.load(file)

input_embeddings_data, input_mask_invert_data, input_masks_data

mask our own masks


In [ ]:
input_embeddings_data

In [ ]:


from transformers import BartForConditionalGeneration

_MODEL = None

def get_model():
    global _MODEL

    if _MODEL is None:
        pretrained_bart = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
        checkpoint_path = '/Users/michaelholborn/Documents/SoftwareLocal/monotropism/thoughtx/task1_task2_taskNRv2_finetune_BrainTranslator_skipstep1_b1_20_30_5e-05_5e-07_unique_sent.pt'  # Change to the path of your model
        _MODEL = BrainTranslator(pretrained_bart)
        model_weights = torch.load(checkpoint_path, map_location=torch.device('cpu'))
        _MODEL.load_state_dict(model_weights)
        _MODEL.eval()

    return _MODEL


In [ ]:

import torch

# Convert loaded data to PyTorch tensors
input_embeddings_tensor = torch.tensor(input_embeddings_data)
input_masks_tensor = torch.tensor(input_masks_data)
input_mask_invert_tensor = torch.tensor(input_mask_invert_data)

input_embeddings_tensor.shape, input_masks_tensor.shape, input_mask_invert_tensor.shape

In [ ]:


from transformers import BartTokenizer, BartForConditionalGeneration
import torch.nn as nn
import torch.nn.functional as F

# Define the BrainTranslator model
class BrainTranslator(nn.Module):
    def __init__(self, pretrained_layers, in_feature=840, decoder_embedding_size=1024, additional_encoder_nhead=8, additional_encoder_dim_feedforward=2048):
        super(BrainTranslator, self).__init__()
        
        self.pretrained = pretrained_layers
        self.additional_encoder_layer = nn.TransformerEncoderLayer(d_model=in_feature, nhead=additional_encoder_nhead, dim_feedforward=additional_encoder_dim_feedforward, batch_first=True)
        self.additional_encoder = nn.TransformerEncoder(self.additional_encoder_layer, num_layers=6)
        self.fc1 = nn.Linear(in_feature, decoder_embedding_size)

    def forward(self, input_embeddings_batch, input_masks_batch, input_mask_invert, decoder_input_ids):
        encoded_embedding = self.additional_encoder(input_embeddings_batch, src_key_padding_mask=input_mask_invert)
        encoded_embedding = F.relu(self.fc1(encoded_embedding))
        out = self.pretrained(inputs_embeds=encoded_embedding, attention_mask=input_masks_batch, decoder_input_ids=decoder_input_ids, return_dict=True)
        return out

# Initialize model
pretrained_bart = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
model = BrainTranslator(pretrained_bart)

# Create a placeholder token
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
placeholder_token = tokenizer("<s>", return_tensors="pt")

# Attempt a forward pass
with torch.no_grad():
    try:
        outputs = model(input_embeddings_tensor, input_masks_tensor, input_mask_invert_tensor, placeholder_token["input_ids"])
        result = "Forward pass successful!"
    except Exception as e:
        result = str(e)

In [ ]:
generated_ids = outputs.logits.argmax(dim=-1)
generated_ids

In [4]:
result
outputs
# Extract the generated token IDs from the model's outputs
generated_ids = outputs.logits.argmax(dim=-1)
generated_ids

# Decode the token IDs to text
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
generated_text


'ij'